In [3]:
##
## ---------------------------------------------------------------------------
##

# Import SparkSession
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType, FloatType, StructType, StructField, IntegerType,  StringType, ArrayType

## the distance function
from scipy.spatial import distance
## creates the feature vector
from pyspark.ml.feature import VectorAssembler
## import numpy
import numpy as np

from pyspark.sql.functions import max, abs

from pyspark.sql.functions import greatest

import time

from pyspark.sql.functions import udf, lit, expr
from pyspark.sql.types import *

import pyspark.sql.functions as F

from statistics import mode

import matplotlib.pyplot as plt

from functools import reduce

from pyspark.ml.linalg import Vectors

import numpy as np

import heapq


## start session
spark = SparkSession.builder.appName("SparkLAESAKnn").getOrCreate()




##DEFINES A UDF FROM L2DIST THE DATAFRAME FROM A QUERY SET
distance_udf = F.udf(lambda x,y: float(distance.euclidean(x, y)), DoubleType())

def simpleF(oq):
    return F.udf(lambda x: float(distance.euclidean(x, oq)), DoubleType())


## start session
spark = SparkSession.builder.appName("SparkRandomCoordinatesDataset").getOrCreate()

##DEFINES A UDF FROM L2DIST THE DATAFRAME FROM A QUERY SET
distance_udf = F.udf(lambda x,y: float(distance.euclidean(x, y)), DoubleType())

##DEFINES A UDF FROM L2DIST THE DATAFRAME FROM A QUERY POINT
distance_udf_2 = F.udf(lambda x: float(distance.euclidean(x, oq)), DoubleType())


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/08 09:58:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/08 09:58:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/08 09:58:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [9]:
## defines query object and 


oq = [(1.0,[0.241,0.845])]
oq1 = [0.241,0.845]
#oq = [(1.0,[0.0,0.0]),(2.0,[1.0,1.0]),(3.0,[0.5,0.5])]
oqColumns = ["idOq","fvOq"]
oqDF = spark.createDataFrame(data=oq, schema = oqColumns)
oqDF.show()
## defines neighbrs amount
k = 10

## defines number of pivots
pivots = 2

+----+--------------+
|idOq|          fvOq|
+----+--------------+
| 1.0|[0.241, 0.845]|
+----+--------------+



In [116]:
##
## DF
##

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("coord_x", FloatType(), True),
    StructField("coord_y", FloatType(), True)])

df = spark.read.csv("/home/weber/Documents/coordDF100.csv",schema=schema)        
size = df.count()
##SMALL SANITY CHECK - @PRODUCTION TESTAR LOADING
df = df.na.drop()

## define the struct for the dimensional feature vector
cNames = df.columns
cNames.remove("id")
assembler = VectorAssembler(
    inputCols=cNames,
    outputCol="fv")

## appends the fv into the dataframe as column
df = assembler.transform(df)

#df.show()


rows_oqDF = oqDF.collect()
# creates random pivots
pivots_list = [(i+1,[np.random.rand(1)[0],np.random.rand(1)[0]]) for i in range(pivots)]
## calculates the distance from oi to pivots
for i in range(len(pivots_list)):
    df = df.withColumn(f'distances_oi_pivot{i+1}', simpleF(pivots_list[i][1])(F.col('fv'))) 

for oq in rows_oqDF: 
    ##
    ## DISTANCE FROM OQ TO PIVOT
    ##
    lista_columns = []
    for i in range(len(pivots_list)):
        df = df.withColumn(f"lower_oq_{int(oq.idOq)}_pivot_{i+1}", \
            abs (distance.euclidean(oq.fvOq, pivots_list[i][1]) - \
            F.col(f'distances_oi_pivot{i+1}')))

        lista_columns.append(f"lower_oq_{int(oq.idOq)}_pivot_{i+1}")

    df = df.withColumn(f"lower_bound_oq_{int(oq.idOq)}", greatest(*[col_name for col_name in lista_columns]))
df.show(2)

+---+-------+-------+--------------------+-------------------+-------------------+--------------------+-------------------+-------------------+
| id|coord_x|coord_y|                  fv|distances_oi_pivot1|distances_oi_pivot2|  lower_oq_1_pivot_1| lower_oq_1_pivot_2|   lower_bound_oq_1|
+---+-------+-------+--------------------+-------------------+-------------------+--------------------+-------------------+-------------------+
| 85|0.76609|0.64468|[0.76608997583389...|0.34417663504013285| 0.3849982369566117|  0.3054486591391675|0.19588167440633134| 0.3054486591391675|
| 86|0.26943|0.14308|[0.26943001151084...| 1.0269985855269559|  0.598427782361222|  0.3773732913476555|0.40931121981094165|0.40931121981094165|
| 87|0.14337|0.68237|[0.14337000250816...|  0.789380875048398| 0.2514871152481183| 0.13975558086909767|0.06237055269783795|0.13975558086909767|
| 88|0.50937|0.54759|[0.50937002897262...| 0.5609518337460507|0.21701895533635088| 0.08867346043324964|0.02790239278607054|0.08867346043

In [111]:
def laesa_join(df, oqDF, k):  
    r_laesa = float('inf')
    h = 0
    count = df.count()
    stop_iteration = True
    dict_results = {}
    for oq in rows_oqDF: 
        
        df_oq = df.orderBy(df[f"lower_bound_oq_{int(oq.idOq)}"])
        def process_partition(iterator):
            nonlocal r_laesa, h, stop_iteration 
            global_pq = []
            for row in iterator:
                if stop_iteration:
                    h += 1
                    if len(global_pq) < k:
                        heapq.heappush(global_pq, (-(distance.euclidean(oq.fvOq, row.fv)), row.id))
                    else:
                        nextDist = -(distance.euclidean(oq.fvOq, row.fv))
                        if (nextDist > global_pq[0][0]): 
                            heapq.heappush(global_pq, (nextDist, row.id))
                            heapq.heappop(global_pq) 
                            r_laesa = global_pq[0][0]
                    if k < h < count and row[f"lower_bound_oq_{int(oq.idOq)}"] >= -(r_laesa):
                        stop_iteration = False
            yield global_pq, count-h
        rdd = df_oq.rdd.mapPartitions(process_partition)
        pq = rdd.flatMap(lambda x: x).collect()
    
        dict_results[oq.idOq] = pq
        
    
    # SORT ELEMENTS AND RETURN TO POSITIVE VALUES
    
    final_dict = {}

    for key, value in dict_results.items():
        new_lst = [(-x[0], x[1]) for x in value[0]]
        sorted_lst = sorted(new_lst, key=lambda x: x[0])
        final_dict[key] = [sorted_lst, value[1]]
    
    return final_dict

In [112]:
def knn(dataframe, oq, k):       
    
    ##CALCULATES THE DIST FROM ELEMENTS WITHIN THE DF TO THE QUERY POINT
    df = dataframe.withColumn('distances', simpleF(oq)(F.col('fv')))
    
    ##SORTS AND RETRIEVES THE TOP-K RESULTS
    resultSet = df.select("id", "distances").orderBy('distances').limit(k)
    resultSet.show()
    return(resultSet)

In [ ]:
def bridk(df, oqDF, k):
        
    ##CALCULATES THE DIST FROM ELEMENTS WITHIN THE DF TO THE QUERY POINT
    df = df.withColumn('distances', simpleF(oq1)(F.col('fv'))).orderBy('distances')    
    
    # auxiliary variables
    list_result = []
    h = 1
    dropped = 0
    first_run = True
    
    # collect rows from df 
    rows = df.collect()
    
    # for each row in sorted df
    for row in rows:
        
        #get first element 
        if first_run:
            id_influence = row["id"]
            point_influence = row["fv"]
            r_influence = row["distances"]
            list_result.append((row["id"],row["distances"]))
            first_run =  False
            continue
            
        distanceToR = distance.euclidean(point_influence,row["fv"])
    
        if distanceToR > r_influence:

            #GET ELEMENT
            id_influence = row["id"]
            point_influence = row["fv"]
            r_influence = row["distances"]

            list_result.append((row["id"],row["distances"]))
            h += 1
        else:
            dropped += 1
        
        if h == k:
            break
    
    return (list_result, dropped) 

In [123]:
result = bridk_simple(df, oqDF, 20)
print(f"Lista de elementos: {result[0]}")
print(f"Elementos droppados: {result[1]}")

Lista de elementos: [(72, 0.05820743921144072), (38, 0.12999364072773112), (10, 0.1344279099442985), (62, 0.13982856560515264), (27, 0.14495558725385918), (19, 0.1485328999366544), (11, 0.15564100109966134), (95, 0.1573325997174506), (58, 0.16704707890477422), (59, 0.17396759359291186), (50, 0.18737019353054477), (87, 0.18968429302669815), (31, 0.2041673574810173), (69, 0.2414419383409781), (90, 0.28516281362122187), (89, 0.29474754862481345), (81, 0.3292871796962782), (97, 0.33385168022244444), (16, 0.37956609394209706), (13, 0.4510565038299228)]
Elementos droppados: 10
